In [6]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

import lab9_lib

In [7]:
MU = 5
LAMBDA = 20
MUTATION_PROB = 0.2

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 1_500

In [8]:
def generate_random_individual():
    ind = np.random.choice([0, 1], size=LENGTH_SOLUTION)
    return ind

def mutate(ind, size=1):
    if random.random() < MUTATION_PROB:
        index = np.random.choice(list(range(len(ind))), size=size, replace=False)
        ind[index] = 1 - ind[index]
    return ind

def reproduce(ind1, ind2):
    new_ind = np.ndarray(shape=ind1.shape)
    for i in range(len(ind1)):
        gene_giver = random.choice([ind1, ind2])
        new_ind[i] = gene_giver[i]
    return new_ind

In [13]:
def ga(fitness, parents, parents_evals, memoization=False):
    if memoization:
        pop_history = {}
        for i in range(len(parents)):
            pop_history[parents[i].tobytes()] = parents_evals[i]

    for generation in tqdm(range(NUMBER_GENERATIONS)):
        offsprings = []
        offsprings_evals = []
        while len(offsprings) < LAMBDA:
            if (max(parents_evals)-min(parents_evals)) > 1e-5:
                probabilities = [(score-min(parents_evals)) / (max(parents_evals)-min(parents_evals)) for score in parents_evals]
            else:
                probabilities = [1 for score in parents_evals]
            probabilities = np.array(probabilities)/sum(probabilities)
            p1 = random.choices(parents, k=1, weights=probabilities)[0]
            p2 = random.choices(parents, k=1, weights=probabilities)[0]
            new_ind1 = mutate(reproduce(p1, p2))
            if memoization and pop_history.get(new_ind1.tobytes()) != None:
                continue
            new_ind1_eval = fitness(new_ind1)
            offsprings.append(new_ind1)

            offsprings_evals.append(new_ind1_eval)
            if memoization:
                pop_history[new_ind1.tobytes()] = new_ind1_eval

        all_people = parents + offsprings
        all_evals = parents_evals + offsprings_evals
        best_people = np.argsort(all_evals)[::-1]

        parents = []
        parents_evals = []
        for i in range(MU):
            parents.append(all_people[best_people[i]])
            parents_evals.append(all_evals[best_people[i]])
        if np.max(parents_evals) - 1.0 >= 0:
            print(f"Early stopping {generation=}")
            break

    return parents, parents_evals

Without memoization

In [14]:
fitness = lab9_lib.make_problem(2)
parents = [generate_random_individual() for _ in range(50)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals = ga(fitness, parents, parents_evals, memoization=False)
i_best = np.argmax(parents_evals)
print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

100%|██████████| 1500/1500 [00:29<00:00, 50.07it/s]

[0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.

With memoization

In [15]:
fitness = lab9_lib.make_problem(2)
parents = [generate_random_individual() for _ in range(50)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals = ga(fitness, parents, parents_evals, memoization=True)
i_best = np.argmax(parents_evals)
print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

 31%|███▏      | 469/1500 [00:36<01:21, 12.71it/s]

Early stopping generation=469
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1.